In [9]:
import resource
import os, psutil
from sys import getsizeof
import multiprocessing

In [4]:
s_s = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024
c_s = resource.getrusage(resource.RUSAGE_CHILDREN).ru_maxrss / 1024

s_s, c_s

(1051.18359375, 34.890625)

In [9]:
getsizeof(object())

16

In [ ]:
objs = []

In [10]:
for i in range(5000000):
    objs.append(object())

In [11]:
s_e = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024
c_e = resource.getrusage(resource.RUSAGE_CHILDREN).ru_maxrss / 1024

s_e, c_e

(1051.18359375, 34.890625)

In [12]:
def get_process_memory():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024


def _work(flag):
    s_s = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024
    c_s = resource.getrusage(resource.RUSAGE_CHILDREN).ru_maxrss / 1024
    objs = []
    print("########1", s_s, c_s, get_process_memory())
    for i in range(10 * 1024**2):
        objs.append(object())
    s_e = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024
    c_e = resource.getrusage(resource.RUSAGE_CHILDREN).ru_maxrss / 1024
    print("########2", s_e, c_e, get_process_memory())
    print(flag, "pid[%d]" % os.getpid(), "self:", s_e - s_s, "child:", c_e - c_s)

def _self():
    _work('0')
    
    # chilren
    multiprocessing.Process(target=_work, args=('1',)).start()
    
    import time
    time.sleep(2)
    
    # self
    _work('3')
    
multiprocessing.Process(target=_self, args=()).start()

########1 67.6015625 0.0 67.609375
########2 309.15625 0.0 309.15625
0 pid[1884] self: 241.5546875 child: 0.0
########1 68.0859375 0.0 68.09375
########2 308.69921875 0.0 308.69921875
1 pid[1891] self: 240.61328125 child: 0.0
########1 309.15625 0.0 71.52734375
########2 309.15625 0.0 309.15234375
3 pid[1884] self: 0.0 child: 0.0
